In [16]:
!/Users/zhouyang/Downloads/google-cloud-sdk/bin/gcloud auth list

     Credentialed Accounts
ACTIVE  ACCOUNT
*       yang.zhou.yzu@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [17]:
!/Users/zhouyang/Downloads/google-cloud-sdk/bin/gcloud services enable texttospeech.googleapis.com

In [22]:
!/Users/zhouyang/Downloads/google-cloud-sdk/bin/gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=UozaMteqF4x3tPDgKXbbTuD13uo1gU&access_type=offline&code_challenge=RlkN28gtx4z3GHFe95h0alQN3U1RwAsqyAtILQX-uNY&code_challenge_method=S256


Credentials saved to file: [/Users/zhouyang/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "genuine-segment-253508" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owni

In [11]:
!pip install ipython google-cloud-texttospeech

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 140.0 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 594.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 915.5 kB/s eta 0:00:00a 0:00:01
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 1.3 MB/s eta 0:00:0000:0100:010m
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/12

In [23]:
from typing import Sequence

import google.cloud.texttospeech as tts


def unique_languages_from_voices(voices: Sequence[tts.Voice]):
    language_set = set()
    for voice in voices:
        for language_code in voice.language_codes:
            language_set.add(language_code)
    return language_set


def list_languages():
    client = tts.TextToSpeechClient()
    response = client.list_voices()
    languages = unique_languages_from_voices(response.voices)

    print(f" Languages: {len(languages)} ".center(60, "-"))
    for i, language in enumerate(sorted(languages)):
        print(f"{language:>10}", end="\n" if i % 5 == 4 else "")
        

In [24]:
list_languages()

---------------------- Languages: 57 -----------------------
     af-ZA     ar-XA     bg-BG     bn-IN     ca-ES
    cmn-CN    cmn-TW     cs-CZ     da-DK     de-DE
     el-GR     en-AU     en-GB     en-IN     en-US
     es-ES     es-US     eu-ES     fi-FI    fil-PH
     fr-CA     fr-FR     gl-ES     gu-IN     he-IL
     hi-IN     hu-HU     id-ID     is-IS     it-IT
     ja-JP     kn-IN     ko-KR     lt-LT     lv-LV
     ml-IN     mr-IN     ms-MY     nb-NO     nl-BE
     nl-NL     pa-IN     pl-PL     pt-BR     pt-PT
     ro-RO     ru-RU     sk-SK     sr-RS     sv-SE
     ta-IN     te-IN     th-TH     tr-TR     uk-UA
     vi-VN    yue-HK

In [25]:
import google.cloud.texttospeech as tts


def text_to_wav(voice_name: str, text: str):
    language_code = "-".join(voice_name.split("-")[:2])
    text_input = tts.SynthesisInput(text=text)
    voice_params = tts.VoiceSelectionParams(
        language_code=language_code, name=voice_name
    )
    audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

    client = tts.TextToSpeechClient()
    response = client.synthesize_speech(
        input=text_input,
        voice=voice_params,
        audio_config=audio_config,
    )

    filename = f"{voice_name}.wav"
    with open(filename, "wb") as out:
        out.write(response.audio_content)
        print(f'Generated speech saved to "{filename}"')
        

Generated speech saved to "en-US-News-K.wav"


In [29]:
import google.cloud.texttospeech as tts


def list_voices(language_code=None):
    client = tts.TextToSpeechClient()
    response = client.list_voices(language_code=language_code)
    voices = sorted(response.voices, key=lambda voice: voice.name)

    print(f" Voices: {len(voices)} ".center(60, "-"))
    for voice in voices:
        languages = ", ".join(voice.language_codes)
        name = voice.name
        gender = tts.SsmlVoiceGender(voice.ssml_gender).name
        rate = voice.natural_sample_rate_hertz
        print(f"{languages:<8} | {name:<24} | {gender:<8} | {rate:,} Hz")
        
list_voices("en")

----------------------- Voices: 105 ------------------------
en-AU    | en-AU-Neural2-A          | FEMALE   | 24,000 Hz
en-AU    | en-AU-Neural2-B          | MALE     | 24,000 Hz
en-AU    | en-AU-Neural2-C          | FEMALE   | 24,000 Hz
en-AU    | en-AU-Neural2-D          | MALE     | 24,000 Hz
en-AU    | en-AU-News-E             | FEMALE   | 24,000 Hz
en-AU    | en-AU-News-F             | FEMALE   | 24,000 Hz
en-AU    | en-AU-News-G             | MALE     | 24,000 Hz
en-AU    | en-AU-Polyglot-1         | MALE     | 24,000 Hz
en-AU    | en-AU-Standard-A         | FEMALE   | 24,000 Hz
en-AU    | en-AU-Standard-A         | FEMALE   | 24,000 Hz
en-AU    | en-AU-Standard-B         | MALE     | 24,000 Hz
en-AU    | en-AU-Standard-B         | MALE     | 24,000 Hz
en-AU    | en-AU-Standard-C         | FEMALE   | 24,000 Hz
en-AU    | en-AU-Standard-C         | FEMALE   | 24,000 Hz
en-AU    | en-AU-Standard-D         | MALE     | 24,000 Hz
en-AU    | en-AU-Standard-D         | MALE     | 24,00

In [48]:
text_to_wav("en-US-Wavenet-I", """
Hi everyone, the paper that I’m going to present is titled.

Synthesizing Speech Test Cases with Text-to-Speech?. An Empirical Study on the False Alarms in Automated Speech Recognition Testing.

We find that there are up to 52% of failed test cases found by state-of-the-art speech testing tools are false alarms.

I’m Zhou Yang, a PhD student at Singapore Management University. This is a collaborative work done by researchers at SMU and Monash Malaysia.

Automated Speech Recognition, as suggested by its name, tries to automatically convert a piece of audio, into transcripts

The ASR techniques have been widely adopted in our daily life. 
For example, Hey Siri. 
We can also input text using our voice.
We have 3 papers about autonomous driving in this session.
ASR systems can also be deployed in cars to recognize voice commands like asking your car to navigate to a certain location.

Typically there will be a “however” if a presenter say so many good things.

So, however, A-S-R is not perfect.
One offensive example is that the YouTube auto caption system inappropriately transcribes audio for kids into offensive contents.
I won’t read it out. So I just give you 10 seconds and you can read in mind. 

Another example is that researchers shows that actually low-quality ASR in car cause more cognitive distractions to drivers, which may cause serious accidents.

So, it is important to test ASR systems.
The test case consists of two parts, audio, and its corresponding ground truth transcripts.

Depending on what you have, there are two ways to create speech test cases manually.
If you have audio, for example, many records of conversation, just ask people to transcribe.
If you have text, for example, a lot of books, just ask people to read it out.

However, this manual process can be rather expensive.
In some cases, it’s not like oh I ask a group of people to read the content.
Assuming that we want to test ASR’s ability in recognizing medical audio, we may need experts to label the data, which can be costly.
This motivates us to synthesize speech test cases automatically.

One way is to use transformations to synthesize audio.
Given the original audio, we can apply a combination of different transformations, like add background noise, change the volume, add perturbation, change the speed, and many others.
After the transformations, we obtain new audio.
There is a metamorphic relationship here.
The audio after slight transformation, sounds almost the same to human, so ASR should recognize them as the same transcripts!

Another way is to automate the test case generation process is to use the text-to-speech technique.
In the manual testing process, 
Text is converted into audio, and an ASR system transcribes the audio. 
The oracle is that the original text and the transcriptions should be the same.

If we ask another person to read the text out, the relationship still holds.

Another person, still holds.

Then, what about asking another AI model, called text-to-speech model, to convert the text into audio, and let the ASR to transcribe.
This new metamorphic relationship forms the foundation of using TTS system to automatically generate speech test cases.

But a potential problem is that, the generated audio itself can be invalid. In other words, a TTS model cannot pronounce texts correctly.

In this case, if an ASR doesn’t correctly transcribe the generated audio, we cannot decide whether it is because the ASR is not functional or it is because the TTS is not functional.

Previous works use the framework of differential testing to mitigate the problem.
There are many ASR systems available.
We feed the generated audio into multiple ASR systems, if at least one of them can correctly transcribe, it means that the audio is valid.
In this case, if an ASR system cannot produce the correct transcription, we find a failed test case.

But if all the ASR systems cannot correctly transcribe, we still cannot decide which part of this pipeline has problem. 
So we call such audio as interminable test case.

We tried to use this pipeline to test 4 ASR systems.
Given the same budget, which means that the same number of generated audio, we find thousands of failed test cases for them.
The results look promising. 
We can find so many failures automatically in a cheap way, as generate audio using TTS is generally much cheaper than human labeling.

Again, when we say good things, there is a However.
However, a question to ask is that does the synthetic failures reflect actual failures. 
As there are differences between the audio pronounced by humans and TTS systems, in terms of different aspects like …
Maybe an ASR system cannot recognize TTS-generated audio, but it can correctly recognize human speech.
We call this case as “false alarms.”
And we care about how many false alarms we have when testing ASR systems using TTS.





""")

str2='''

In this empirical study, we use two widely used datasets that come with both text and audio.
We only use texts for generating test cases, and the audio is only used to evaluate false alarms.
We experiment with 5 popular ASR systems that demonstrate state-of-the-art performance, and 4 TTS system, covering both commercial ones like Google TTS and open-source ones.

The first question we try to explore is that “How notable is the performance difference between ASR systems when transcribing human audio and TTS-generated audio?”
We use a metric called “average word error rate” to measure ASR performance on a set of test cases.
The rightmost figures show performance on human audio.
We can observe that in general, performance on audio generated by GlowTTS and Google TTS are closer to that on human audio.
While ASR performance on Espeak and Festival generated audio is far worse than the performance on human audio.

This lead to another question, How prevalent are false alarms when using TTS-generated audio to test ASR systems?
In the figure, we use the false alarm rate as a metric, meaning the ratio of false alarms in all the failed test cases.

From this figure, we can first observe that false alarms are common in the ASR testing scenario!
When testing Wav2Vec model using the Espeak, over 50% of failed test cases are false alarms.

We can also observe that the choice of TTS to generate audio makes a difference in false alarms.
Using Google TTS to generate audio produce the least number of false alarms across all the ASR systems and 2 datasets.

GoogleTTS appears to be the most effective speech synthesizer!

Then, we manually sample false alarms and listen to them to summarize some patterns that cause false alarms.
We find that in some cases, ASR systems cannot pronounce consonants prominently.
For example, it may pronounce “officers” as “offices”
ASR systems may not pronounce suffixes clearly.
Besides, they may not pronounce grammatical clearly.
For example, they easily mis pronounce “as” to “is”.
 

 In some case, TTS systems pronounce in a correct way.
But ASR systems fail to recognize.
For example, ASRs fail to transcribe some homophones or fail to transcribe words with multiple pronunciations.

Since there are many false alarms, we wonder whether we can build a predictor to find the texts that are more likely to be false alarms in advance.

So we split all the test cases into 3 sets: 60% training data, 10% validation data, and 30% testing data.
Each text will produce 20 pieces of audio, as we have 4 TTS systems and 5 ASR systems.
If over 10 audio are false alarms, we label it as 1.
Otherwise, we label it as 0.

Then, we train an RNN-based estimator that takes the text as input, and produce the proability that this text leads to false alarms.
On the testing set, we find that the estimator performs well. It have over 93% precision and over 97% F-1 score.


In conclusion, this empirical study shows that using synthetic audio to test ASR can result in many false alarms.
Choosing high-quality TTS (like me) can better estimate ASR performance.
This paper also trains an RNN-based estimator to predict false alarms.
In the future, the authors believe that it is promising to extend the work to more ASR and TTS systems.


Hope it inspires you.
You may wonder how this lazy guy standing near the desk can use TTS to answer questions.
The truth is that he cannot.
So ask as much as you can. 
Any questions are welcome.'''

Generated speech saved to "en-US-Wavenet-I.wav"


In [49]:
text_to_wav("en-US-Wavenet-I", str2)

Generated speech saved to "en-US-Wavenet-I.wav"


In [75]:
text_to_wav("en-US-Studio-O", '''Hope it inspires you.
You may wonder how this lazy guy standing near the desk can use TTS to answer questions.
The truth is that he cannot.
So just smash him with as many questions as you can. 
Any questions are welcome.
'''
)

Generated speech saved to "en-US-Studio-O.wav"
